# LawChat

**Use natural language to query a library of previosuly cited legal cases**

We make use of LangChain for:

- Document Loaders: WebBaseLoader
- Text Splitter: TokenTextSplitter
- Vector Store: Cassandra
- Models: OpenAIEmbeddings and ChatOpenAI
- Chain: FLARE

We will be using the Australasian Legal Information Institute as our source legislation and court judgments ("case law").

## Configuration

**Create your `.env1` file**

1. Copy the .env.example file to `.env`
2. Specify your Astra and openAI parameters.

In [3]:
import os
from dotenv import load_dotenv

"""
class Settings():
    load_dotenv()
    ASTRA_DB_KEYSPACE = os.environ['ASTRA_DB_KEYSPACE']
    ASTRA_DB_SECURE_BUNDLE_PATH = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
    ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']
    OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
"""

ASTRA_DB_KEYSPACE = os.environ['ASTRA_DB_KEYSPACE']
ASTRA_DB_SECURE_BUNDLE_PATH = os.environ['ASTRA_DB_SECURE_BUNDLE_PATH']
ASTRA_DB_APPLICATION_TOKEN = os.environ['ASTRA_DB_APPLICATION_TOKEN']
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Astra DB Connectivity

In [5]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.cqlengine import connection


# load settings and keys
#settings = Settings


def getCluster():
    """
    Create a Cluster instance to connect to Astra DB.
    Uses the secure-connect-bundle and the connection secrets.
    """
    cloud_config = {"secure_connect_bundle": ASTRA_DB_SECURE_BUNDLE_PATH}
    auth_provider = PlainTextAuthProvider("token", ASTRA_DB_APPLICATION_TOKEN)
    return Cluster(cloud=cloud_config, auth_provider=auth_provider)


def get_astra():
    """
    This function is used by LangChain Vectorstore.
    """
    cluster = getCluster()
    astraSession = cluster.connect()
    return astraSession, ASTRA_DB_KEYSPACE


def initSession():
    """
    Create the DB session and return it to the caller.
    Most important, the session is also set as default and made available
    to the object mapper through global settings. I.e., no need to actually
    do anything with the return value of this function.
    """
    cluster = getCluster()
    session = cluster.connect()
    session.set_keyspace("lawchat")
    connection.register_connection("my-astra-session", session=session)
    connection.set_default_connection("my-astra-session")
    return connection

#### Define the Vector Store

In [6]:
from langchain.vectorstores import Cassandra
from langchain.embeddings.openai import OpenAIEmbeddings

# define Embedding model
embeddings = OpenAIEmbeddings()

# Set up the vector store
print("Setup Vector Store")
session, keyspace = get_astra()
vectorstore = Cassandra(
    embedding=embeddings,
    session=session,
    keyspace=keyspace,
    table_name="nswsc",
)

Setup Vector Store


## Load library data

We are using data from the Australasian Legal Information Institute. AustLII maintains collections of primary materials: legislation and court judgments ("case law"). 

For this project, we are sourcing case law from the Supreme Court of New South Wales.

#### Utility functions

In [7]:
import re

"""
Function to clean text from web pages
"""
def clean_text(text: str):
    # Normalize line breaks to \n\n (two new lines)
    text = text.replace("\r\n", "\n\n")
    text = text.replace("\r", "\n\n")

    # Replace two or more spaces with a single space
    text = re.sub(" {2,}", " ", text)

    # Remove leading spaces before removing trailing spaces
    text = re.sub("^[ \t]+", "", text, flags=re.MULTILINE)

    # Remove trailing spaces before removing empty lines
    text = re.sub("[ \t]+$", "", text, flags=re.MULTILINE)

    # Remove empty lines
    text = re.sub("^\s+", "", text, flags=re.MULTILINE)

    return text


In [8]:
import tiktoken

"""
Function to calculate the number of tokens in a text string.
"""

encoding = tiktoken.get_encoding("cl100k_base")

def num_tokens_from_string(string: str) -> int:
    num_tokens = len(encoding.encode(string))
    return num_tokens

#### Get data files

We load a number of HTML pages using the LangChain WebBaseLoader. Each of those pages contains lots of superfluous content so we extract only the relevant article context.

In [9]:
from langchain.document_loaders import WebBaseLoader

urls = [
    "https://www.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/423.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/2002/949.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/4.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/2005/1181.html",
    "https://www8.austlii.edu.au/cgi-bin/viewdoc/au/cases/nsw/NSWSC/1998/483.html"
    ]
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
}

print("Loading Data")
url_loaders = WebBaseLoader(urls, header_template=headers)
data = url_loaders.load()

"""
Extract only the actual Article content from the web page and clean
"""
print("Cleaning Data")
for i, d in enumerate(data):
    d.page_content = ""
    source = d.metadata['source']
    thedoc = WebBaseLoader(source, header_template=headers).scrape()
    # extract only the Article content from the web page
    td = thedoc.find("article", {"class": "the-document"}).text
    d.page_content = clean_text(td)
    data[i] = d

print (f"Number of documents: {len(urls)}")
print (f"Number of tokens: {num_tokens_from_string(data[0].page_content)}")


Loading Data
Cleaning Data
Number of documents: 5
Number of tokens: 33772


#### Split the data into chunks

In [10]:
from langchain.text_splitter import TokenTextSplitter

CHUNK_SIZE = 200

# Chunk the data
print("Splitting Data")
text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=50)
docs = text_splitter.split_documents(data)
print(f"Number of chunks: {len(docs)}")

Splitting Data
Number of chunks: 673


#### Store data and embeddings in Astra DB

In [11]:
print("Adding texts to Vector Store")
texts, metadatas = zip(*((doc.page_content, doc.metadata) for doc in docs))
vectorstore.add_texts(texts=texts, metadatas=metadatas)

Adding texts to Vector Store


['68fa1f46079c4b0bb41d4a6d53ef0a8d',
 '344679873ced4e43966016a63a297f3c',
 'd8988ed5bc414764a4db9aea4de671b0',
 'e554eb519057450c9536a7422dc81d63',
 '1152a09483204877b4d1929026f0b60b',
 'f879823e0e99482c9a6dd4ede70f62c2',
 '74e6bbacb89d4121bee00fccc7257444',
 '7a2b623ee7074b379f1edd5bd44b43d2',
 '0cec821aafd149ecbb73a68365eb0fdb',
 '9d45e780a60742558d5dcc94f21c30c9',
 '94ee314b4abc4d9cb54a718e4785de42',
 '99c191057d2e405785ced95b7d234c6d',
 'bb0bb960575b498fb9fd9ec60b6019cf',
 'be03ff9a840947e7bafe1d756e153129',
 '47e2823acbed4df2847bdc53187c7d90',
 'cd827f9dde464e749bc33a33bdb4469f',
 '4bf93c412a8248a8a10d49c49ad0c87e',
 'fa6eb45b15f54d6f9fc32e095511b63c',
 'b3ab5e541be34215897178dc62795284',
 '1ce6bfd9daec4fcd921332951e47f54c',
 'c58d067088634843a6a2d2fe856bd919',
 '3df1f9525848458b89b7f0a85b1934c5',
 'ed636ce36a944a279d8252f388c0e3c0',
 '772ebb094f2f4fec873f346d14ce1093',
 'e4177fb7b35e45a3bf56559875832c56',
 '099e8ed050784ae0b2ae9764e6cda1f3',
 '2cf54aff5a0b4e2f8c7742cd680da142',
 

## Query

#### Define the Retriever
by setting "k" in kwargs, we can control how many results are retrieved from the vector store and passed into the Flare chain.

- Ideally, we should not provide a "K", but the only way I can get this to work is limit the K above to a single result in order to not breach the token limit of 4096

In [14]:
import sys
from langchain.chains import FlareChain
from langchain.chat_models import ChatOpenAI

"""
Due to the default FLARE chain having the openAI model hardcoded 
to a 4096 token model, we have to limit the number of tokens. 
To do that we use k=1.
"""
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
#retriever = vectorstore.as_retriever()

#### Define FLARE chain

Appears that the model attribute is ignored as an error is thrown if more than 4096 tokens are supplied.

- the only way I can get this to work is to either:

    - limit the K above to a single result in order to not breach the token limit. But that means we are not retreiving enough context from the data.
    - reduce the chunk size to create smaller chunks and thus reduce the number of tokens passed to the LLM.

In [15]:
flare = FlareChain.from_llm(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    retriever=retriever,
    max_generation_len=164,
    min_prob=0.3,
)

# Define your query
query = "List the orders from the case AMALGAMATED vs MARSDEN"

flare_result = flare.run(query)

print(f"QUERY: {query}\n\n")
print(f"FLARE RESULT:\n    {flare_result}\n\n")

QUERY: List the orders from the case AMALGAMATED vs MARSDEN


FLARE RESULT:
     The orders from the case Amalgamated Television Services v Marsden Matter No 40005/97 [1998] NSWSC 4 (4 February 1998) were that the appeal was allowed in part. 




#### Use OpenAI LLM for comparison of not using FLARE

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI()
llm_result = llm(query)
print(f"LLM RESULT:\n    {llm_result}\n\n")